In [1]:
from Attention import Attention

Using TensorFlow backend.
/home/viole/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/viole/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/viole/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/viole/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [2]:
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense, Embedding, Activation

from keras.utils import normalize

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_csv('Combined Reviews.csv')

In [6]:
df.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [7]:
df['Review'].value_counts()

Works great.                                                                                                                                           2
I love this place.                                                                                                                                     2
Great Phone.                                                                                                                                           2
Don't buy this product.                                                                                                                                2
Great phone.                                                                                                                                           2
                                                                                                                                                      ..
I ordered this for sony Ericsson W810i but I think it only worked once (thats when

In [8]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [9]:
df.shape

(1986, 2)

In [10]:
t = Tokenizer()
t.fit_on_texts(df['Review'])
text_mat = t.texts_to_sequences(df['Review'])

def get_tokenizer(tokenizer=t):
    
    return tokenizer

In [11]:
def find_vocab_size(matrix = text_mat):
    
    sequence_count = [set(text_mat[i])for i in range(len(text_mat))]
    total = []
    for i in range(len(sequence_count)):
        for j in sequence_count[i]:
            total.append(j)
    
    print(len(set(total))) #total number of unique words, or vocabulary of the text
    
    vocab = len(set(total))
    return vocab

In [12]:
len_mat = []
for i in range(len(text_mat)):
    len_mat.append(len(text_mat[i]))

In [13]:
from statistics import mean, median
print("Maximum length of sequence : {}".format(max(len_mat)))
print("Average length of sequence : {}".format(mean(len_mat)))
print("Median length of sequence : {}".format(median(len_mat)))

Maximum length of sequence : 32
Average length of sequence : 10.641490433031219
Median length of sequence : 9.0


In [14]:
text_padding = sequence.pad_sequences(text_mat, maxlen=32, padding='post')

In [15]:
text_padding[0].shape

(32,)

In [16]:
def attention_model():
    
#     model = Sequential()
#     model.add(Embedding(find_vocab_size()+1, 128))
#     model.add(LSTM(128, return_sequences=True, recurrent_dropout=0.3, dropout=0.25))
#     model.add(Attention())
#     model.add(Dense(1))
#     model.add(Activation('sigmoid'))
    
#     model.summary()

    from keras.layers import Input
    
    inputs = Input((text_padding[0].shape))
    x = Embedding(find_vocab_size()+1, 128)(inputs)
    att_in = LSTM(128, return_sequences=True, recurrent_dropout=0.3, dropout=0.3)(x)
    att_out = Attention()(att_in)
    outputs = Dense(1, activation='sigmoid', trainable =True)(att_out)
    model = Model(inputs, outputs)
    
    model.summary()
    
    model.compile(optimizer='adam', metrics=['accuracy'], loss='binary_crossentropy')
    
    return model

In [17]:
attn_model = attention_model()

3258
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 32, 128)           417152    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32, 128)           131584    
_________________________________________________________________
attention_1 (Attention)      (None, 128)               160       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 549,025
Trainable params: 549,025
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [18]:
features, target = text_padding, df['Sentiment']

In [19]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', patience=5)
ckpt = ModelCheckpoint('attn_weights.hdf5', save_best_only=True, save_weights_only=True, monitor='val_accuracy')

In [20]:
ep, bs = 20, 16
attn_model.fit(features,target, validation_split=0.2, epochs=ep, batch_size=bs, callbacks=[es, ckpt])
NAME  = 'attn_lstm_model_{}_ep_{}_bs'.format(ep,bs)
attn_model.save(NAME)


Train on 1588 samples, validate on 398 samples
Epoch 1/20
1588/1588 [==============================] - 5s 3ms/step - loss: 0.6151 - accuracy: 0.6285 - val_loss: 0.5770 - val_accuracy: 0.6960
Epoch 2/20
1588/1588 [==============================] - 4s 3ms/step - loss: 0.3659 - accuracy: 0.8552 - val_loss: 0.5748 - val_accuracy: 0.7186
Epoch 3/20
1588/1588 [==============================] - 4s 3ms/step - loss: 0.1949 - accuracy: 0.9339 - val_loss: 0.4950 - val_accuracy: 0.7889
Epoch 4/20
1588/1588 [==============================] - 4s 3ms/step - loss: 0.1141 - accuracy: 0.9685 - val_loss: 0.7083 - val_accuracy: 0.7538
Epoch 5/20
1588/1588 [==============================] - 4s 3ms/step - loss: 0.0970 - accuracy: 0.9754 - val_loss: 0.7835 - val_accuracy: 0.7462
Epoch 6/20
1588/1588 [==============================] - 4s 3ms/step - loss: 0.0958 - accuracy: 0.9704 - val_loss: 0.6704 - val_accuracy: 0.7538
Epoch 7/20
1588/1588 [==============================] - 4s 2ms/step - loss: 0.0568 - acc

In [64]:
predicted_sent = ["Negative", "Positive"]

def predict_on_test_sentences(model,sent):
    
    t = get_tokenizer()
    test_converted = t.texts_to_sequences([sent])
        
    padded_text = sequence.pad_sequences(test_converted, padding='post', maxlen=32)
    
    assert padded_text.shape[1] == 32  
    
    y_prob = model.predict(padded_text)
    y_classes = y_prob.argmax(axis=-1)
    
    print("Probability : {}".format(y_prob[0][0]))
    
    return predicted_sent[int(round(y_prob[0][0]))]

In [55]:
# sent_to_predict = process_test_sentences("Excellent food.")
# y_prob = attn_model.predict(sent_to_predict) 
# y_classes = y_prob.argmax(axis=-1)

In [56]:
from keras.models import load_model

In [31]:
lstm_model = load_model('base_lstm_model_20_ep_16_bs')

In [65]:
predict_on_test_sentences(lstm_model, "I absolutely love the food here. The service is great")

Probability : 0.9664285778999329


'Positive'

In [66]:
predict_on_test_sentences(attn_model,"I hate the food, bad service. Not even bothered to clean up.")

Probability : 0.0058388919569551945


'Negative'

In [67]:
predict_on_test_sentences(attn_model, "I absolutely love the food here. The service is great")

Probability : 0.9982967972755432


'Positive'

In [68]:
predict_on_test_sentences(lstm_model,"I hate the food, bad service. Not even bothered to clean up.")

Probability : 0.0023912680335342884


'Negative'